In [4]:
import cvxpy as cp
import numpy as np
import pandas as pd

In [19]:
# parameters
initial_balance = 100
max_months = 61 # we're going to look at the beginning of month 61 to evaluate the end of month 60
crops_data = pd.read_csv("crops.csv")
crops_data.index = crops_data['crop']
crops_data

,crop,cost,selling price,time to grow
crop,,,,
peas,peas,10,12,2
potatos,potatos,5,8,4
raspberries,raspberries,15,25,4
saffron,saffron,175,500,6


In [78]:
# goal is to maximize profit by only buying crops

# the real variables you can control
peas = cp.Variable((max_months), "peas") # number of peas to buy at the beginning of month i
potatos = cp.Variable((max_months), "potatos") # number of potatos to buy at the beginning of month i
raspberries = cp.Variable((max_months), "raspberries") # number of raspberries to buy at the beginning of month i
saffron = cp.Variable((max_months), "saffron") # number of saffron to buy at the beginning of month i

# auxilliary variable
grown_peas = cp.Variable((max_months), "grown_peas") # number of peas to buy at the beginning of month i
grown_potatos = cp.Variable((max_months), "grown_potatos") # number of potatos to buy at the beginning of month i
grown_raspberries = cp.Variable((max_months), "grown_raspberries") # number of raspberries to buy at the beginning of month i
grown_saffron = cp.Variable((max_months), "grown_saffron") # number of saffron to buy at the beginning of month i

# technically auxilliary variable, since can be written in terms of the variables you can control, but will optimize the last balance
balance = cp.Variable((max_months), "balance")

# constraints, but written in a form cvxpy won't accept. still helpful to write them all down
constraints = []

# grown peas
for i in range(max_months):
    if i < 2:
        constraints.append(grown_peas[i] == 0)
    else:
        constraints.append(grown_peas[i] == peas[i - 2])

# grown potatos
for i in range(max_months):
    if i < 4:
        constraints.append(grown_potatos[i] == 0)
    else:
        constraints.append(grown_potatos[i] == potatos[i - 4])

# grown raspberries
for i in range(max_months):
    if i < 4:
        constraints.append(grown_raspberries[i] == 0)
    else:
        constraints.append(grown_raspberries[i] == raspberries[i - 4])

# grown saffron
for i in range(max_months):
    if i < 6:
        constraints.append(grown_saffron[i] == 0)
    else:
        constraints.append(grown_saffron[i] == saffron[i - 6])

# balance constraints
for i in range(max_months):
    cost = crops_data['cost']['peas']*peas[i] + crops_data['cost']['potatos']*potatos[i] + crops_data['cost']['raspberries']*raspberries[i] + crops_data['cost']['saffron']*saffron[i]
    revenue = crops_data['selling price']['peas']*grown_peas[i] + crops_data['selling price']['potatos']*grown_potatos[i] + crops_data['selling price']['raspberries']*grown_raspberries[i] + crops_data['selling price']['saffron']*grown_saffron[i]
    previous_balance = (i == 0 and initial_balance) or balance[i-1]
    constraints.append(balance[i] == previous_balance - cost + revenue)
    constraints.append(balance[i] >= 0)

# max balance[max_months]
obj = cp.Maximize(balance[max_months - 1])

prob = cp.Problem(obj, constraints)
optimum = prob.solve(solver=cp.CLARABEL)

print("optimal value: %s\n" % optimum)
for variable in prob.variables():
    print("Variable %s: value %s" % (variable.name(), variable.value))

optimal value: inf

Variable balance: value None
Variable grown_peas: value None
Variable peas: value None
Variable grown_potatos: value None
Variable potatos: value None
Variable grown_raspberries: value None
Variable raspberries: value None
Variable grown_saffron: value None
Variable saffron: value None
